In [1]:
import random
import string
import numpy as np
import pandas as pd
from tabulate import tabulate

### Config

In [2]:
# How many dishes are available and in what range of calories
DISH_COUNT = 300
MIN_CALORIES_PER_DISH = 100
MAX_CALORIES_PER_DISH = 1500

# User config - how many days, how many dishes per day, how many calories per day
DAYS_PER_MENU = 7
DISHES_PER_DAY = 3
CALORIES_PER_DAY = 2000
# Penalty for dishes repaeating during the week
PENALTY = 400

# Genetic algorithm config
POPULATION_SIZE = 20
MATING_POPULATION_SIZE = 10
ITERATIONS = 3000

#### Helper functions

In [16]:
def visualize_menu(menu):
    print("Score: " + str(score_menu(menu)))
    headers = ["Day", "Calories", "Dishes"]
    calories = count_calories_per_day(menu)
    table = []
    for day in range(DAYS_PER_MENU):
        table.append([day, calories[day], menu[day]])
    print(tabulate(table, headers, tablefmt="fancy_grid"))

#### Real data

In [17]:
dishes = {}
data = pd.read_csv('data.csv', delimiter=',', encoding='utf-8', low_memory=False)
df = pd.DataFrame(data, columns=['name', 'Food Group', "calories_portion"])

while len(dishes) != DISH_COUNT:
    random_choice = df.sample()
    if MIN_CALORIES_PER_DISH < random_choice.calories_portion.values[0] < MAX_CALORIES_PER_DISH:
        dishes[random_choice.name.values[0]] = random_choice.calories_portion.values[0]
dishes = list(dishes.items())
print(*dishes[:7], sep='\n')

('Sausage Pork And Beef With Cheddar Cheese Smoked', 227.92)
('Veal Shoulder Blade Chop Separable Lean Only Cooked Grilled', 135.15)
('Beef Australian Imported Wagyu Loin Top Loin Steak/roast Boneless Separable Lean Only Aust. Marble Score 4/5 Raw', 254.22)
('Potato Pudding', 287.28)
('Pears Canned Heavy Syrup Drained', 148.74)
('Beef Round Top Round Separable Lean And Fat Trimmed To 1/8 Inch Fat Choice Cooked Braised', 212.5)
('Restaurant Latino Arepa (Unleavened Cornmeal Bread)', 214.62)


#### Sample random menu

In [18]:
def create_random_menu():
    menu = np.ndarray((DAYS_PER_MENU, DISHES_PER_DAY), tuple)
    for day in range(DAYS_PER_MENU):
        for dish_index in range(DISHES_PER_DAY):
            menu[day, dish_index] = random.choice(dishes)
    return menu

In [27]:
def count_calories_per_day(menu):

    calories = []
    for day in range(DAYS_PER_MENU):
        calories.append(0)
        for dish_index in range(DISHES_PER_DAY):
            calories[-1] += menu[day, dish_index][1]
    
    return calories

def count_duplicates_per_day(menu):
    
    names = np.reshape(menu, DAYS_PER_MENU * DISHES_PER_DAY)
    unique, counts = np.unique(names, return_counts=True)
    names_dict = dict(zip(unique, counts))

    attr = []
    for day in range(DAYS_PER_MENU):
        attr.append(0)
        for dish_index in range(DISHES_PER_DAY):
            attr[-1] += (names_dict[menu[day, dish_index]] - 1 )
    
    print(attr)
    return attr

def score_menu(menu):
    
    differences = [calories - CALORIES_PER_DAY for calories in count_calories_per_day(menu)]
    penalty = [(attr*PENALTY) ** 2 for attr in count_duplicates_per_day(menu)]
    squares = [calories ** 2 for calories in differences]
    
    return sum(squares + penalty)

In [35]:
sample_menu = create_random_menu()
visualize_menu(sample_menu)

[1, 0, 0, 0, 1, 0, 0]
Score: 10684233.0637
╒═══════╤════════════╤═════════════════════════════════════════════════════════════════════════════════════════════════╕
│   Day │   Calories │ Dishes                                                                                          │
╞═══════╪════════════╪═════════════════════════════════════════════════════════════════════════════════════════════════╡
│     0 │     651.97 │ [('Potatoes French Fried Steak Fries Salt Added In Processing Frozen Oven-Heated', 196.84)      │
│       │            │  ('Beef New Zealand Imported Variety Meats And By-Products Tongue Raw', 274.59)                 │
│       │            │  ('Frankfurter Or Hot Dog Sandwich Fat Free Plain On Whole Grain White Bun', 180.54)]           │
├───────┼────────────┼─────────────────────────────────────────────────────────────────────────────────────────────────┤
│     1 │     764.88 │ [('Bread Salvadoran Sweet Cheese (Quesadilla Salvadorena)', 205.7)                     

In [10]:
def crossover(first_menu, second_menu):
    child = first_menu.copy()
    for day_index in range(DAYS_PER_MENU):
        if random.choice((0, 1)) == 1:
            child[day_index] = second_menu[day_index]
    return child


def random_mutate(menu):
    menu[random.randint(0, DAYS_PER_MENU - 1), random.randint(0, DISHES_PER_DAY - 1)] = random.choice(dishes)
    return menu


def swap_mutate(menu):
    first_day_index = random.randint(0, DAYS_PER_MENU - 1)
    second_day_index = random.randint(0, DAYS_PER_MENU - 1)
    dish_index = random.randint(0, DISHES_PER_DAY - 1)
    menu[first_day_index, dish_index], menu[second_day_index, dish_index] = menu[second_day_index, dish_index], menu[
        first_day_index, dish_index]
    return menu

In [11]:
population = []
for _ in range(POPULATION_SIZE):
    population.append(create_random_menu())

In [12]:
for i in range(ITERATIONS):
    population.sort(key=score_menu)
    mating_population = population[:MATING_POPULATION_SIZE]

    offspring_population = []
    for _ in range(POPULATION_SIZE - MATING_POPULATION_SIZE):
        offspring_population.append(crossover(random.choice(mating_population), random.choice(mating_population)))
        if random.random() < 0.1:
            offspring_population[-1] = random_mutate(offspring_population[-1])
        if random.random() < 0.1:
            offspring_population[-1] = swap_mutate(offspring_population[-1])

    population = mating_population + offspring_population

In [13]:
def check_duplicant(menu):
    names = np.reshape(menu, 21)
    unique, counts = np.unique(names, return_counts=True)
    is_repetable = dict(zip(unique, counts))
    
    for elem in is_repetable:
        if is_repetable[elem] > 2:
            print("duplicants detected")
            return
    print("No duplicants")

    
population.sort(key=score_menu)
visualize_menu(population[0])    
check_duplicant(population[0])

Score: 332.8300000000001
╒═══════╤════════════╤═══════════════════════════════════════════════════════════════════════════════════════════════════════════════════╕
│   Day │   Calories │ Dishes                                                                                                            │
╞═══════╪════════════╪═══════════════════════════════════════════════════════════════════════════════════════════════════════════════════╡
│     0 │    2000.05 │ [('Restaurant Mexican Cheese Enchilada', 666.12)                                                                  │
│       │            │  ('Poultry Mechanically Deboned From Mature Hens Raw', 551.61)                                                    │
│       │            │  ('Pie Crust Standard-Type Frozen Ready-To-Bake Enriched Baked', 782.32)]                                         │
├───────┼────────────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ 